In [237]:
# =============================================
#
# load cifar-10-small and project down
#
 
from tqdm import tqdm
import numpy as np
    
def unpickle( file ):
    import cPickle
    fo = open(file, 'rb')
    dict = cPickle.load(fo)
    fo.close()
    return dict
 
data = unpickle( 'cifar-10-batches-py/data_batch_1' )
 
features = data['data']
labels = data['labels']
labels = np.atleast_2d( labels ).T
 
N = 1000
D = 10
 
# only keep N items
features = features[ 0:N, : ] 
labels = labels[ 0:N, : ]
 
# project down into a D-dimensional space
features = np.dot( features, np.random.randn( 3072, D) )
 
# whiten our data - zero mean and unit standard deviation
features = (features - np.mean(features, axis=0)) / np.std(features, axis=0)



W = np.random.randn(features.shape[1],features.shape[1])


NUM_EPOCHS = 1000
step_size = 0.1

print features.shape
print W.shape
print labels.shape

def loss_function(W):
    loss = []
    for i in range(features.shape[0]):
        scores = np.atleast_2d(W.dot(features[i]))

        scores -= np.max(scores,axis=1)

        scores = np.exp(scores)

        scores /= np.sum(scores)
        loss.append(-1*np.log(scores[0][labels[i]]))
        
    return loss
        
#     scores = W.dot(features.T).T
# #     print '1', scores.shape
# #     print '2',np.max(scores, axis=1)[0]
# #     print '3',np.max(scores, axis=1).shape
# #     print '3.1',scores[0]
#     scores = np.transpose(scores.T - np.max(scores, axis=1))
# #     print '4',scores[0]
#     scores = np.exp(scores)
# #     print '5',scores[0]
# #     print '6',scores.shape

#     scores = np.transpose(scores.T / np.sum(scores, axis=1))
# #     print '7',scores[0]


# #     print np.take(scores, labels)[0]
# #     print labels[0]
    


#     loss = -np.log(np.take(scores, labels))
    
#     loss = np.sum(loss)
# #     print loss
#     return loss
    
    
    

def numerical_gradient(loss_function,W):
    loss_return = loss_function(W)

    epsilon = .0001

    print loss_return
    print loss_function(W+ epsilon)
    print (loss_function(W + epsilon) - loss_return ) / epsilon 

    
    grad = (loss_function(W + epsilon) - loss_return ) / epsilon 

    return loss_return, grad
        


for i in range(NUM_EPOCHS):
    
    loss_function_value, grad = numerical_gradient( loss_function, W )
#     print grad
#     print W
    W = W - step_size * grad
#     print W
    break
#     if i == 2:
#         break
    
    

(1000, 10)
(10, 10)
(1000, 1)
[array([ 3.78486922]), array([ 7.42017651]), array([ 7.29722019]), array([ 6.63267971]), array([ 4.05031349]), array([ 8.18439904]), array([ 5.66494511]), array([ 5.03165195]), array([ 2.35886648]), array([ 11.28203057]), array([ 4.90901059]), array([ 4.07937053]), array([ 7.74655376]), array([ 3.7897375]), array([ 2.5954741]), array([ 6.87420281]), array([ 4.45360336]), array([ 0.37340887]), array([ 8.17020404]), array([ 4.2849371]), array([ 2.70072947]), array([ 0.29232153]), array([ 1.98220604]), array([ 3.6337723]), array([ 7.41041916]), array([ 1.79390217]), array([ 6.48642489]), array([ 8.62855072]), array([ 3.33292509]), array([ 4.98574557]), array([ 4.94178312]), array([ 7.12829526]), array([ 6.37593891]), array([ 1.4117685]), array([ 1.30641659]), array([ 2.98948566]), array([ 0.60040692]), array([ 5.9094497]), array([ 0.02496679]), array([ 0.01938413]), array([ 1.00976591]), array([ 2.83393897]), array([ 2.7175786]), array([ 3.26084428]), array([

[array([ 3.78486922]), array([ 7.42017651]), array([ 7.29722019]), array([ 6.63267971]), array([ 4.05031349]), array([ 8.18439904]), array([ 5.66494511]), array([ 5.03165195]), array([ 2.35886648]), array([ 11.28203057]), array([ 4.90901059]), array([ 4.07937053]), array([ 7.74655376]), array([ 3.7897375]), array([ 2.5954741]), array([ 6.87420281]), array([ 4.45360336]), array([ 0.37340887]), array([ 8.17020404]), array([ 4.2849371]), array([ 2.70072947]), array([ 0.29232153]), array([ 1.98220604]), array([ 3.6337723]), array([ 7.41041916]), array([ 1.79390217]), array([ 6.48642489]), array([ 8.62855072]), array([ 3.33292509]), array([ 4.98574557]), array([ 4.94178312]), array([ 7.12829526]), array([ 6.37593891]), array([ 1.4117685]), array([ 1.30641659]), array([ 2.98948566]), array([ 0.60040692]), array([ 5.9094497]), array([ 0.02496679]), array([ 0.01938413]), array([ 1.00976591]), array([ 2.83393897]), array([ 2.7175786]), array([ 3.26084428]), array([ 5.29216044]), array([ 4.36782

TypeError: unsupported operand type(s) for -: 'list' and 'list'